### 練習問題2.79
⼆つの数値の等価性をテストするジェネリックな等価性述語 equ? を定義し、<br/>
ジェネリック算術演算パッケージに組み込め。この演算は、通常の数値、有理数、複素数に対して動作しなければならない。

In [1]:
(define (lookup key table)
  (let ((record (assoc key (cdr table))))
    (if record (cdr record)
        #f)
    )
  )

(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (insert! key value table)
  (let ((record (assoc key (cdr table))))
    (if record (set-cdr! record value)
        (set-cdr! table (cons (cons key value) (cdr table)))
        )
    )
  'ok)

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

;(define (make-table) (list '*table*))

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))

; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (cons type-tag contents))

; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum))
  )
; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum))
  )

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types: APPLY-GENERIC"
           (list op type-tags ))))))

In [2]:
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))
(define (equ? x y) (apply-generic 'equ? x y))    ;追加手続き 戻り値がTrue/False


In [3]:
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put 'equ? '(scheme-number scheme-number) (lambda (x y) (= x y)))    ;追加手続き 戻り値がTrue/False
  (put 'make 'scheme-number (lambda (x) (tag x)))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [4]:
(define (gcd a b)
  (if (= b 0) a
      (gcd b (remainder a b)))
  )

(define (install-rational-package)
  ;; 内部⼿続き
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (gcd n d))) (cons (/ n g) (/ d g))))
  (define (add-rat x y) (make-rat (+ (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (sub-rat x y) (make-rat (- (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (mul-rat x y) (make-rat (* (numer x) (numer y))
                                  (* (denom x) (denom y))))
  (define (div-rat x y) (make-rat (* (numer x) (denom y)) (* (denom x) (numer y))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational) (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational) (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational) (lambda (x y) (tag (mul-rat x y)))) 
  (put 'div '(rational rational) (lambda (x y) (tag (div-rat x y))))
  (put 'equ? '(rational rational) (lambda (x y) (and (= (numer x) (numer y)) (= (denom x) (denom y))))) ;追加手続き
  (put 'make 'rational (lambda (n d) (tag (make-rat n d))))
  'done)

(define (make-rational n d) ((get 'make 'rational) n d))

(install-rational-package)

done

In [49]:
(import "math")
;(define (make-from-mag-ang r a)
;  (define (dispatch op)
;    (cond ((eq? op 'real-part) (* r (math.cos a)))
;          ((eq? op 'imag-part) (* r (math.sin a)))
;          ((eq? op 'magnitude) r)
;          ((eq? op 'angle) a)
;          (else (error "Unknown op: MAKE-FROM-MAG-ANG" op))
;          )
;  )
;dispatch)

(define (square x)(* x x))

(define (install-rectangular-package)
  ;; 内部⼿続き
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z) (sqrt (+ (square (real-part z)) (square (imag-part z)))))
  (define (angle z) (math.atan2 (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a) (cons (* r (math.cos a)) (* r (math.sin a))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular (lambda (r a) (tag (make-from-mag-ang r a))))
  ;(put 'equ? '(rectangular rectangular) (lambda (x y) (and (= (real-part x) (real-part y)) (= (imag-part x) (imag-part y))))) ;追加手続き
'done)

(define (install-polar-package)
  ;; 内部⼿続き
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z) (* (magnitude z) (math.cos (angle z))))
  (define (imag-part z) (* (magnitude z) (math.sin (angle z))))
  (define (make-from-real-imag x y) (cons (sqrt (+ (square x) (square y))) (math.atan2 y x)))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar (lambda (r a) (tag (make-from-mag-ang r a))))
  ;(put 'equ? '(polar polar) (lambda (x y) (and (= (magnitude x) (magnitude y)) (= (angle x) (angle y))))) ;追加手続き
  'done)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))

(install-rectangular-package)
(install-polar-package)

done

In [50]:
(define (install-complex-package)
  ;; 直交形式パッケージと極形式パッケージからインポートした⼿続き
  (define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))
  ;; 内部⼿続き
  (define (add-complex z1 z2) (make-from-real-imag (+ (real-part z1) (real-part z2)) (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2) (make-from-real-imag (- (real-part z1) (real-part z2)) (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2) (make-from-mag-ang (* (magnitude z1) (magnitude z2)) (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2) (make-from-mag-ang (/ (magnitude z1) (magnitude z2)) (- (angle z1) (angle z2))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex) (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex) (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex) (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex) (lambda (z1 z2) (tag (div-complex z1 z2))))
  ;(put 'equ? '(complex complex) (lambda (z1 z2) (equ? z1 z2))) ;追加手続き
  ;(put 'equ? '(complex complex) (lambda (x y) (and (= (real-part x) (real-part y)) (= (imag-part x) (imag-part y))))) ;追加手続き
  (put 'equ? '(complex complex) (lambda (x y) (and (= (magnitude x) (magnitude y)) (= (angle x) (angle y))))) ;追加手続き
  (put 'make-from-real-imag 'complex (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (make-complex-from-real-imag x y) ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a) ((get 'make-from-mag-ang 'complex) r a))

(install-complex-package)

done

In [51]:
(define a (make-scheme-number 2.0))
(define b (make-scheme-number 2.00))
(define c (make-scheme-number 2.01))

(display a)
(display "\n")
(display b)
(display "\n")

(display (equ? a b))
(display "\n")

(display (equ? a c))
(display "\n")


(scheme-number . 2.0)
(scheme-number . 2.0)
True
False


In [9]:
(define a (make-rational 1 2))
(define b (make-rational -3 -6))
(define c (make-rational 4 6))

(display a)
(display "\n")
(display b)
(display "\n")

(display (equ? a b))
(display "\n")

(display (equ? a c))
(display "\n")


(rational 1 . 2)
(rational 1 . 2)
True
False


(complex rectangular 1 . 2)
(complex rectangular 1.0 . 2.0)
True
False


In [10]:
(define a (make-from-real-imag 1 2))
(define b (make-from-real-imag 1.0 2.0))
(define c (make-from-real-imag 1 3))

(display a)
(display "\n")
(display b)
(display "\n")

(display (equ? a b))
(display "\n")

(display (equ? a c))
(display "\n")


(rectangular 1 . 2)
(rectangular 1.0 . 2.0)
True
False


In [12]:
(define a (make-from-mag-ang 1 2))
(define b (make-from-mag-ang 1.0 2.0))
(define c (make-from-mag-ang 1 3))

(display a)
(display "\n")
(display b)
(display "\n")

(display (equ? a b))
(display "\n")

(display (equ? a c))
(display "\n")


(polar 1 . 2)
(polar 1.0 . 2.0)
True
False


In [37]:
(define a (make-complex-from-real-imag 1 2))
(define b (make-complex-from-real-imag 1.0 2.0))
(define c (make-complex-from-real-imag 1 3))

(display a)
(display "\n")
(display b)
(display "\n")

(display (equ? a b))
(display "\n")

(display (equ? a c))
(display "\n")


(complex rectangular 1 . 2)
(complex rectangular 1.0 . 2.0)
True
False


In [52]:
(define a (make-complex-from-real-imag -1.0 0.0))
(define b (make-complex-from-mag-ang 1.0 math.pi))
;(define b (make-complex-from-real-imag -1.0 0))



In [53]:
(display a)
(display "\n")
(display b)
(display "\n")


(complex rectangular -1.0 . 0.0)
(complex polar 1.0 . 3.141592653589793)


In [54]:
(display (equ? a b))
(display "\n")


True


In [55]:
(display math.pi)

3.141592653589793

In [56]:
(add a b)

(complex rectangular -2.0 . 1.2246467991473532e-16)

In [46]:
(sub a b)

(complex rectangular 0.0 . -1.2246467991473532e-16)

In [47]:
(mul a b)

(complex polar 1.0 . 6.283185307179586)

In [48]:
(div a b)

(complex polar 1.0 . 0.0)